In [14]:
#!pip3 install multiprocess
from multiprocessing import Poo
l
import math
import numpy as np
import pandas as pd
# !pip3 install tqdm

import matplotlib.pyplot as plt
from tqdm import tqdm

ImportError: cannot import name 'Poo' from 'multiprocessing' (/usr/lib/python3.8/multiprocessing/__init__.py)

In [15]:
def arrangement_1(nleds,d):

    l=math.sqrt(nleds);

    ledcord=np.zeros((2,nleds));

    temp=0;
    for k in range(nleds):
        ledcord[0,k]=-(l-1)*d/2 +((k)%l)*d;
        if (k)%l ==0 and k != 0:
            temp=temp+1;
        ledcord[1,k]=-(l-1)*d/2 + temp*d;

    return ledcord

def E_xy_cent(ledcord,nleds,Ir_0,z,d):
    
    x=0;
    y=0;
    E=0;
    for i in range(nleds):
        den=np.square(np.square(x-ledcord[0,i]) +np.square(y-ledcord[1,i]) +z**2);
        E= E + (z**2 * Ir_0)*(np.reciprocal(den));

    return E;
    
def E_xy_end(ledcord,nleds,Ir_0,z,d,x_end):
    
    x=x_end;
    y=0;
    E=0;
    for i in range(nleds):
        den=np.square(np.square(x-ledcord[0,i]) +np.square(y-ledcord[1,i]) +z**2);
        E= E + (z**2 * Ir_0)*(np.reciprocal(den));

    return E;

def plot_E_xy_full(ledcord,nleds,Ir_0,z,d):
    l=math.sqrt(nleds)
    x = np.arange(-10, 10, 0.01)
    y = np.arange(-10, 10, 0.01)

    x,y=np.meshgrid(x, y, sparse=False, indexing='ij')

    E=np.zeros(np.size(x));

    for i in range(nleds):
        den=np.square(np.square(x-ledcord[0,i]) +np.square(y-ledcord[1,i]) +z**2);
        E= E + (z**2 * Ir_0)*(np.reciprocal(den));
        
    return E;

#     s=surface(x,y,E);
#     xlabel('x-coordinate(m)');
#     ylabel('y-coordinate(m)');
#     zlabel('Irradinace(W/m^2)');
#     s.EdgeColor = 'none';


In [16]:
nleds=49;
Ir_0 = 1;
d=0.122292;
z=0.05;
ledcord = arrangement_1(nleds,d);
x_end=0.0;
print(E_xy_end(ledcord,nleds,Ir_0,z,d,x_end))

450.9825479895467


In [ ]:
Ir_0 = 1;

# print(plot_E_xy_full(ledcord,nleds,Ir_0,z,d))
# E_cent=E_xy_cent(ledcord,nleds,Ir_0,z,d)
# E_half = E_cent/2
# E_99 = E_cent * 0.99
# x_half = 0
# x_99 = 0
# x_half_flag = False
# x_99_flag = False

final_list = []


def ratio(d):
#     for d in np.linspace(0.01,0.2,38):
    for nleds in [1,9,25,49,81,121,169]:
        ledcord = arrangement_1(nleds,d)
        for z in np.linspace(0.05,0.5,90):
            E_cent = E_xy_cent(ledcord,nleds,Ir_0,z,d)
            E_half = E_cent/2
            E_99 = E_cent * 0.99
            x_half = 0
            x_99 = 0
            x_half_flag = False
            x_99_flag = False

            for x_end in np.linspace(0,5,10000):



                E_end=E_xy_end(ledcord,nleds,Ir_0,z,d,x_end)

                if E_end < E_half and not x_half_flag:
                    x_half = x_end
                    x_half_flag = True

                if E_end < E_99 and not x_99_flag:
                    x_99 = x_end
                    x_99_flag = True

            final_list.append([nleds,d,z,E_cent,E_99, x_99,E_half,x_half])
    return final_list

with Pool(7) as p:
      l = list(tqdm(p.imap(ratio,np.linspace(0.01,0.2,38)), total=38))
# pool=Pool();
# l=pool.map(ratio,np.linspace(0.01,0.2,10))
# #     ratio(d)

 26%|██▋       | 10/38 [5:22:37<21:31:47, 2768.14s/it]

In [ ]:
t=[]
for i in l:
    t.extend(i)
final_list=t

In [ ]:
df = pd.DataFrame(final_list,columns=["nleds","d","z","E_cent","E_99","x_99","E_half","x_half"])

In [ ]:
df["ratio"]=df["x_99"]/df["x_half"]

In [ ]:
len(df["E_half"].unique())

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

In [ ]:
plt.plot(df["ratio"])

In [ ]:
len(df["ratio"].unique())

In [ ]:
# ratio_max=max(df["ratio"])


In [ ]:
idx = df.groupby(['nleds'])['ratio'].transform(max) == df['ratio']
df[idx]

In [ ]:
ef=df[df["ratio"] == df["ratio"].max()]
max(ef["E_cent"])